## TF Files

In [0]:
from random import shuffle
import glob
import sys
import cv2
import numpy as np
import tensorflow as tf

In [2]:
tf.__version__

'1.14.0'

In [0]:
# tf.enable_eager_execution()

In [4]:
#write a function to download the dataset from tenserflow datasets
from tensorflow.keras.datasets import cifar10
from keras.utils import np_utils
from sklearn.utils import shuffle

def get_data(x,preprocess=False):
  (X_train, y_train), (X_test, y_test) = x.load_data()
  num_classes = len(np.unique(y_train))
  X_train, y_train = shuffle(X_train, y_train)
  X_test, y_test = shuffle(X_test, y_test)

  if preprocess:
        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        X_train /= 255
        X_test /= 255
  
#   Y_train = np_utils.to_categorical(y_train, num_classes)
#   Y_test = np_utils.to_categorical(y_test, num_classes)
  return X_train, y_train, X_test, y_test

Using TensorFlow backend.


In [5]:
X_train, Y_train, X_test, Y_test=get_data(cifar10)
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

170500096/170498071 [==============================] - 11s 0us/step


In [0]:
#takes an image and resizes to one size and changes color to RGB(Rohan gave his ref when his val acc was stuck becoz he forgot to do BGR2RGB)
def load_image(addr):
    addr = str(addr)
    img = cv2.imread(addr)
    if img is None:
        return None
    img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [0]:
import tensorflow as tf
import numpy as np

def _bytes_feature(value):
	"""Returns a bytes_list from a string / byte."""
	return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
	"""Returns a float_list from a float / double."""
	return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
	"""Returns an int64_list from a bool / enum / int / uint."""
	return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def serialize_function(x, y):
	example = tf.train.Example(features=tf.train.Features(
					feature={
							'image': _bytes_feature(tf.compat.as_bytes(x.tostring())),
							'label': _int64_feature(int(y))
					}))
	return example.SerializeToString()


def convert_to_tfrecord(data_set, file_name):
	with tf.python_io.TFRecordWriter(file_name) as record_writer:
		for x, y in data_set:
			if isinstance(x, (np.ndarray, np.generic)) and isinstance(y, (np.ndarray, np.generic)):
				record = serialize_function(x, y)
			else:
				record = serialize_function(x.numpy(), y.numpy())
			record_writer.write(record)
		print(f"TFRecord id created at path : '{file_name}', Done")


def parser(record, shape=[32, 32, 3], num_of_class=10):
	feature_description = {
	'image': tf.FixedLenFeature([], tf.string, default_value=''),
	'label': tf.FixedLenFeature([], tf.int64, default_value=0)
	# 'feature2': tf.FixedLenFeature([], tf.string, default_value=''),
	# 'feature3': tf.FixedLenFeature([], tf.float32, default_value=0.0),
		}
	parsed = tf.parse_single_example(record, feature_description)
	image = tf.decode_raw(parsed["image"], tf.uint8)
	image = tf.cast(image, tf.float32)
	image = tf.reshape(image, shape=shape)
	label = tf.cast(parsed["label"], tf.int32)
	label = tf.one_hot(label, num_of_class, dtype=tf.int32)
	return image, label


def input_foo(filenames, buffer_size, seed, batch_size, GPU_buffer_size):
	dataset = tf.data.TFRecordDataset(filenames=filenames, num_parallel_reads=40)

	dataset = dataset.apply(
		tf.contrib.data.shuffle_and_repeat(buffer_size=buffer_size,
										   seed=seed)
		)
	dataset = dataset.apply(
		tf.contrib.data.map_and_batch(
								  map_func=parser,
								  batch_size=batch_size,
								  num_parallel_calls=tf.data.experimental.AUTOTUNE)
		)
	dataset = dataset.apply(tf.contrib.data.prefetch_to_device('/GPU:0', buffer_size=GPU_buffer_size))
#     dataset = dataset.map(parser, num_parallel_calls=12)
#     dataset = dataset.batch(batch_size=1000)
#     dataset = dataset.prefetch(buffer_size=2)
	return dataset

class CreateTFRecord(object):
    """docstring for CreateTFRecord"""
    '''
    EXP.
    data_to_write = zip(x_train, y_train)
    file name = 'train.tfrecord'
    '''
    def __init__(self, data_to_write, file_name):
        super().__init__()
        self.data_to_write = data_to_write
        self.file_name = file_name
        convert_to_tfrecord(self.data_to_write, self.file_name)

class Get_dataset(object):
	def __init__(self, file_name , buffer_size=1024, seed=1, batch_size=32, GPU_buffer_size = None):
		self.file_name = file_name
		self.buffer_size=buffer_size
		self.seed=seed
		self.batch_size=batch_size
		self.GPU_buffer_size = GPU_buffer_size
		input_foo(self.file_name, self.buffer_size, self.seed, self.batch_size, self.GPU_buffer_size)
# CreateTFRecord("kk.tfrecord", "kk")

In [8]:
Get_dataset('test.tfrecords')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.data.experimental.shuffle_and_repeat(...)`.
Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take ca

In [9]:
CreateTFRecord(zip(X_test, Y_test), 'test.tfrecords')

TFRecord id created at path : 'test.tfrecords', Done


In [10]:
CreateTFRecord(zip(X_train, Y_train), 'train.tfrecords')

TFRecord id created at path : 'train.tfrecords', Done


In [11]:
!ls -lh

total 179M
drwxr-xr-x 1 root root 4.0K Aug 27 16:17 sample_data
-rw-r--r-- 1 root root  30M Sep 10 14:34 test.tfrecords
-rw-r--r-- 1 root root 150M Sep 10 14:34 train.tfrecords


In [0]:
def parser(record, shape=[32, 32, 3], num_of_class=10):
	feature_description = {
	'image': tf.FixedLenFeature([], tf.string, default_value=''),
	'label': tf.FixedLenFeature([], tf.int64, default_value=0)
	# 'feature2': tf.FixedLenFeature([], tf.string, default_value=''),
	# 'feature3': tf.FixedLenFeature([], tf.float32, default_value=0.0),
		}
	parsed = tf.parse_single_example(record, feature_description)
	image = tf.decode_raw(parsed["image"], tf.uint8)
	image = tf.cast(image, tf.float32)
	image = tf.reshape(image, shape=shape)
	label = tf.cast(parsed["label"], tf.int32)
	label = tf.one_hot(label, num_of_class, dtype=tf.int32)
	return image, label


def input_foo(filenames, buffer_size=1024, seed=1, batch_size=32, GPU_buffer_size = None):
	dataset = tf.data.TFRecordDataset(filenames=filenames, num_parallel_reads=40)

	dataset = dataset.apply(
		tf.contrib.data.shuffle_and_repeat(buffer_size=buffer_size,
										   seed=seed)
		)
	dataset = dataset.apply(
		tf.contrib.data.map_and_batch(
								  map_func=parser,
								  batch_size=batch_size,
								  num_parallel_calls=tf.data.experimental.AUTOTUNE)
		)
	dataset = dataset.apply(tf.contrib.data.prefetch_to_device('/GPU:0', buffer_size=GPU_buffer_size))
#     dataset = dataset.map(parser, num_parallel_calls=12)
#     dataset = dataset.batch(batch_size=1000)
#     dataset = dataset.prefetch(buffer_size=2)
	return dataset



## TFRecords

In [0]:
# from matplotlib import pyplot as plt
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())


# #take one record out from .tfrecords file so we get an image and a label
# def parser(record):
#     #first say what kind of feature in the file i.e. 
#     keys_to_features = {
#         "image_raw": tf.FixedLenFeature([], tf.string),
#         "label":     tf.FixedLenFeature([], tf.int64)
#     }
#     parsed = tf.parse_single_example(record, keys_to_features)  #read one record from the file
#     image = tf.decode_raw(parsed["image_raw"], tf.uint8) #take image out of it and Convert the data from string back to the numbers: 
#     image = tf.cast(image, tf.float32)  #convert it to float depending on the model need
#     image = tf.reshape(image, shape=[32, 32, 3])
#     label = tf.cast(parsed["label"], tf.int32)  #Label cast too

#     return image, label


# def show(image, label):
#   plt.figure()
#   plt.imshow(image)
#   plt.title(label.numpy().decode('utf-8'))
#   plt.axis('off')

#   show(image, label)

# def input_fn(filenames):
#   print("hello")
  
#   dataset = tf.data.TFRecordDataset(filenames=filenames, num_parallel_reads=40) #this tf.data API,one of the most imp in Tensorflow
# #   for raw_record in dataset.take(10):
# #     print(repr(raw_record))
#   print(type(dataset))
#   dataset=dataset.repeat(1)
#   dataset=dataset.shuffle(1024)
#   dataset=dataset.map(parser)
#   dataset=dataset.batch(128).prefetch(32)
  
# #   dataset = dataset.apply(
# #       tf.contrib.data.shuffle_and_repeat(1024, 1)
# #   )
# #   dataset = dataset.apply(
# #       tf.contrib.data.map_and_batch(parser, 32)  # map takes a python function and applies it to every sample
# #   )
#   #print(tf.shape(dataset))
  
#   #for image, label in dataset.take(2):  #checking whether images decoded properly
#     #show(image, label)
#   #dataset = dataset.map(parser, num_parallel_calls=12)
#   #dataset = dataset.batch(batch_size=1000)
# #   dataset = dataset.prefetch(buffer_size=1)
# #   iterator = dataset.make_one_shot_iterator()
    
#     # Create your tf representation of the iterator
# #   image, label = iterator.get_next()
# #   for data in dataset.take(10):
# #      data.eval()
# #   label = tf.one_hot(label, num_classes, dtype=tf.int32)
# #   return image, label
#   return  dataset


# def train_input_fn():
#     return input_fn(filenames=["train.tfrecords"])

# def test_input_fn():
#     return input_fn(filenames=["test.tfrecords"])


In [0]:
train_iterator = input_foo("train.tfrecords")
val_iterator = input_foo('test.tfrecords')


##Training

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# # Load CIFAR10 Data
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()
# img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# # convert to one hot encoing 
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [22]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
# input = keras.layers.Input(tensor=image)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)









Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [0]:
# model_input = keras.layers.Input(tensor=image)

# #Build your network
# model_output = keras.layers.Flatten(input_shape=(-1, 255, 255, 1))(model_input)
# model_output = keras.layers.Dense(1000, activation='relu')(model_output)

#Create your model




train_model = keras.models.Model(inputs=[input], outputs=[output])

In [24]:
train_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 12)   48          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 12)   0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [0]:
keras.utils.plot_model(model, 'my_first_model.png')

In [0]:
# model = Model(inputs=[input], outputs=[output])
# model.summary()

In [0]:
# x_train_batch, y_train_batch = tf.train.shuffle_batch(
#     tensors=[data.train.images, data.train.labels.astype(np.int32)],
#     batch_size=batch_size,
#     capacity=capacity,
#     min_after_dequeue=min_after_dequeue,
#     enqueue_many=enqueue_many,
#     num_threads=8)


In [0]:
import tensorflow as tf


def cnn_model():
    input_layer = tf.keras.layers.Input(shape=(32, 32, 3))
    use_bias = False

    # Layer 1
    conv = tf.keras.layers.Conv2D(32,
                                  kernel_size=(3, 3),
                                  padding='same',
                                  use_bias=use_bias,
                                  activation=None)(input_layer)
    bn = tf.keras.layers.BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)(conv)
    activation = tf.keras.layers.Activation('relu')(bn)

    conv = tf.keras.layers.Conv2D(32,
                                  kernel_size=(3, 3),
                                  padding='same',
                                  use_bias=use_bias,
                                  activation=None)(activation)
    bn = tf.keras.layers.BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)(conv)
    activation = tf.keras.layers.Activation('relu')(bn)

    max_pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(activation)
    dropout = tf.keras.layers.Dropout(0.2)(max_pool)

    # Layer 2
    conv = tf.keras.layers.Conv2D(64,
                                  kernel_size=(3, 3),
                                  padding='same',
                                  use_bias=use_bias,
                                  activation=None)(dropout)
    bn = tf.keras.layers.BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)(conv)
    activation = tf.keras.layers.Activation('relu')(bn)

    conv = tf.keras.layers.Conv2D(64,
                                  kernel_size=(3, 3),
                                  padding='same',
                                  use_bias=use_bias,
                                  activation=None)(activation)
    bn = tf.keras.layers.BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)(conv)
    activation = tf.keras.layers.Activation('relu')(bn)

    max_pool = tf.keras.layers.MaxPooling2D()(activation)
    dropout = tf.keras.layers.Dropout(0.3)(max_pool)

    # Layer 3
    conv = tf.keras.layers.Conv2D(128,
                                  kernel_size=(3, 3),
                                  padding='same',
                                  use_bias=use_bias,
                                  activation=None)(dropout)
    bn = tf.keras.layers.BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)(conv)
    activation = tf.keras.layers.Activation('relu')(bn)

    conv = tf.keras.layers.Conv2D(128,
                                  kernel_size=(3, 3),
                                  padding='same',
                                  use_bias=use_bias,
                                  activation=None)(activation)
    bn = tf.keras.layers.BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)(conv)
    activation = tf.keras.layers.Activation('relu')(bn)

    max_pool = tf.keras.layers.MaxPooling2D()(activation)
    dropout = tf.keras.layers.Dropout(0.4)(max_pool)

    flatten = tf.keras.layers.Flatten()(dropout)

    # Output layers: separate outputs for the weather and the ground labels
    output = tf.keras.layers.Dense(10,
                                   activation='softmax',
                 
                                   name='output')(flatten)

    return tf.keras.Model(inputs=input_layer, outputs=output)

In [0]:
# image, label = input_fn('train.tfrecords')



In [0]:
# model = train_model

In [27]:
model = cnn_model()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
# keras.utils.plot_model(model, 'my_first_model.png')

In [0]:



model.compile(
    optimizer=tf.keras.optimizers.Adam(),
#     optimizer=Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
#     loss='binary_crossentropy',
    metrics=['accuracy']
)

# cifar_est = tf.keras.estimator.model_to_estimator(keras_model=model)


In [29]:
for i, data in enumerate(train_iterator):
    print(data)
    if i==10:
        break

(<tf.Tensor 'IteratorGetNext:0' shape=(?, 32, 32, 3) dtype=float32>, <tf.Tensor 'IteratorGetNext:1' shape=(?, 10) dtype=int32>)
(<tf.Tensor 'IteratorGetNext_1:0' shape=(?, 32, 32, 3) dtype=float32>, <tf.Tensor 'IteratorGetNext_1:1' shape=(?, 10) dtype=int32>)
(<tf.Tensor 'IteratorGetNext_2:0' shape=(?, 32, 32, 3) dtype=float32>, <tf.Tensor 'IteratorGetNext_2:1' shape=(?, 10) dtype=int32>)
(<tf.Tensor 'IteratorGetNext_3:0' shape=(?, 32, 32, 3) dtype=float32>, <tf.Tensor 'IteratorGetNext_3:1' shape=(?, 10) dtype=int32>)
(<tf.Tensor 'IteratorGetNext_4:0' shape=(?, 32, 32, 3) dtype=float32>, <tf.Tensor 'IteratorGetNext_4:1' shape=(?, 10) dtype=int32>)
(<tf.Tensor 'IteratorGetNext_5:0' shape=(?, 32, 32, 3) dtype=float32>, <tf.Tensor 'IteratorGetNext_5:1' shape=(?, 10) dtype=int32>)
(<tf.Tensor 'IteratorGetNext_6:0' shape=(?, 32, 32, 3) dtype=float32>, <tf.Tensor 'IteratorGetNext_6:1' shape=(?, 10) dtype=int32>)
(<tf.Tensor 'IteratorGetNext_7:0' shape=(?, 32, 32, 3) dtype=float32>, <tf.Tenso

In [30]:
N_EPOCHS = 10
BATCH_SIZE = 32
N_VALIDATION = 10000
N_TRAIN = 50000

model.fit(
    train_iterator,
    epochs=N_EPOCHS,
    steps_per_epoch=1000,
    validation_data=val_iterator,
    validation_steps=N_VALIDATION // BATCH_SIZE

)

Epoch 1/10
1000/1000 [==============================] - 27s 27ms/step - loss: 1.6605 - acc: 0.4255 - val_loss: 1.2352 - val_acc: 0.5552
Epoch 2/10
1000/1000 [==============================] - 22s 22ms/step - loss: 1.1314 - acc: 0.5990 - val_loss: 0.9379 - val_acc: 0.6654
Epoch 3/10
1000/1000 [==============================] - 22s 22ms/step - loss: 0.9482 - acc: 0.6663 - val_loss: 0.8787 - val_acc: 0.6968
Epoch 4/10
1000/1000 [==============================] - 21s 21ms/step - loss: 0.8404 - acc: 0.7081 - val_loss: 0.8047 - val_acc: 0.7286
Epoch 5/10
1000/1000 [==============================] - 21s 21ms/step - loss: 0.7763 - acc: 0.7284 - val_loss: 0.7437 - val_acc: 0.7475
Epoch 6/10
1000/1000 [==============================] - 21s 21ms/step - loss: 0.7223 - acc: 0.7477 - val_loss: 0.6947 - val_acc: 0.7542
Epoch 7/10
1000/1000 [==============================] - 21s 21ms/step - loss: 0.6671 - acc: 0.7653 - val_loss: 0.6106 - val_acc: 0.7871
Epoch 8/10
1000/1000 [==========================

In [0]:
# import os
# test_data = os.path.join('./', "test.tfrecords")
test_input = lambda: input_fn('train.tfrecords')

cifar_est.train(input_fn=test_input, steps=7000)

In [0]:
https://github.com/tensorflow/models/blob/master/tutorials/image/cifar10_estimator/cifar10_main.py
    
    https://androidkt.com/train-keras-model-with-tensorflow-estimators-and-datasets-api/

In [0]:
image, label = train_input_fn()

In [0]:
print(image)
print(label)

In [0]:
model.fit(epochs=1,
          steps_per_epoch=100)

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [37]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')

In [39]:
pip install --upgrade --upgrade-strategy only-if-needed https://github.com/kkrunal77/data_science_utils/tarball/master

     - 10kB 2.0MB/s
     |████████████████████████████████| 2.3MB 2.7MB/s 
  Created wheel for data-science-utils: filename=data_science_utils-0.0.0-cp36-none-any.whl size=3765 sha256=100b69b01ac04ca239adcf44862df7b704f091b8f70e9f3b9f8a33b07360852d
  Stored in directory: /tmp/pip-ephem-wheel-cache-h23u6o9o/wheels/72/1a/9f/4d1aadec4afb9693f42c521c5be90d606f56f435ba21e38756
Successfully built data-science-utils


In [40]:
pip install git+https://github.com/kkrunal77/data_science_utils.git#egg=data_science_utils

In [0]:

from data.imports import *

In [0]:
CreateTFRecord()


In [43]:
?data.d

Object `data.d` not found.
